# Explore Pattern Between Taxi Engagement Time and Neighborhoods in New York City (Data Prepare)

## About Data
The whole practice will use several different data sources.  

**Taxi Trip Data**  
https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page provides "Data of trips taken by taxis and for-hire vehicles in New York City".  "The yellow and green taxi trip records include fields capturing pick-up and drop-off dates/times, pick-up and drop-off locations, trip distances, itemized fares, rate types, payment types, and driver-reported passenger counts".  I'll use pick-up/drop-off dates/times and locations in green taxi trip data for this practice.

**Foursquare Location Data**  
The taxi pick-up/drop-off locations nearby venue info can be queried via Foursquare API.  And the nearby venue info will be used for neighborhood segmentation and clustering.

**Latitude and Longitude of Location**  
The code to request Latitude and longitude information would look something like this:

In [ ]:
import geocoder

g = geocoder.arcgis('Alphabet City, Manhattan')
latitude = g.lat
longitude = g.lng
print('The geograpical coordinate of East Harlem North, Manhattan are {}, {}.'.format(latitude, longitude))

## Data Prepare

First step, import libraries and read data from nyc website.

In [ ]:
import numpy as np # library to handle data in a vectorized manner
np.random.seed(0)

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib.pyplot as plt

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

import seaborn as sns
sns.set()

In [ ]:
trip_data = pd.read_csv('https://nyc-tlc.s3.amazonaws.com/trip+data/yellow_tripdata_2019-12.csv', low_memory=False)
print(trip_data.head())

zone_lookup = pd.read_csv('https://s3.amazonaws.com/nyc-tlc/misc/taxi+_zone_lookup.csv')
print(zone_lookup.head())

Second step, insert Borough and Zone info into trip_data

In [ ]:
PULocation_Borough = []
PULocation_Zone = []
DOLocation_Borough = []
DOLocation_Zone = []

for t in trip_data.PULocationID:
    PULocation_Borough.append(zone_lookup[zone_lookup['LocationID'] == t].Borough.values[0])
    PULocation_Zone.append(zone_lookup[zone_lookup['LocationID'] == t].Zone.values[0])
for t in trip_data.DOLocationID:
    DOLocation_Borough.append(zone_lookup[zone_lookup['LocationID'] == t].Borough.values[0])
    DOLocation_Zone.append(zone_lookup[zone_lookup['LocationID'] == t].Zone.values[0])

trip_data.insert(loc=6,column='PU_Borough',value=PULocation_Borough)
trip_data.insert(loc=7,column='PU_Zone',value=PULocation_Zone)
trip_data.insert(loc=9,column='DO_Borough',value=DOLocation_Borough)
trip_data.insert(loc=10,column='DO_Zone',value=DOLocation_Zone)

trip_data.head()

Third step, get latitude and longitude data for each taxi zone.

In [ ]:
import geocoder

taxi_zone_location_list = []
for i in range(len(zone_lookup)):
    row = str(zone_lookup.iloc[i].Zone) + ', ' + zone_lookup.iloc[i].Borough 
    taxi_zone_location_list.append(row)

taxi_zone_location_dic = {'location':taxi_zone_location_list}
taxi_zone_location = pd.DataFrame(data=taxi_zone_location_dic)
print(taxi_zone_location.head())

lat = []
lng = []
for i in range(len(taxi_zone_location)):
    row = taxi_zone_location.iloc[i].location + ', New York City, NY'
    xy = geocoder.arcgis(row)
    lat.append(xy.lat)
    lng.append(xy.lng)
taxi_zone_location.insert(loc=1,column='Latitude',value=lat)
taxi_zone_location.insert(loc=2,column='Longitude',value=lng)

# Save the location data into a csv file, so it can be used later.
taxi_zone_location.to_csv('taxi_zone_location.csv', index=False)

Fourth step, prepare venues data.

In [ ]:
CLIENT_ID = # your Foursquare ID
CLIENT_SECRET = # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

In [ ]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        # retry 10 times if request returns null
        for i in range(0,10):
            results = requests.get(url).json()
            if results["response"]['groups'] is not None:
                #print('not None')
                #print(results["response"])
                break
            #print('None')
        results = results["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [ ]:
import time

taxi_zone_venues = getNearbyVenues(names=taxi_zone_location['location'],
                                   latitudes=taxi_zone_location['Latitude'],
                                   longitudes=taxi_zone_location['Longitude']
                                  )

taxi_zone_venues.to_csv('taxi_zone_venues_500.csv', index=False)

Fifth step, cluster venues by neighborhood.

In [ ]:
taxi_zone_venues.groupby('Neighborhood').count()
print('There are {} uniques categories.'.format(len(taxi_zone_venues['Venue Category'].unique())))

In [ ]:
# one hot encoding
taxi_zone_onehot = pd.get_dummies(taxi_zone_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
taxi_zone_onehot['Neighborhood'] = taxi_zone_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [taxi_zone_onehot.columns[-1]] + list(taxi_zone_onehot.columns[:-1])
taxi_zone_onehot = taxi_zone_onehot[fixed_columns]

taxi_zone_onehot.head()

In [ ]:
taxi_zone_grouped = taxi_zone_onehot.groupby('Neighborhood').mean().reset_index()
taxi_zone_grouped.head()

In [ ]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [ ]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = taxi_zone_grouped['Neighborhood']

for ind in np.arange(taxi_zone_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(taxi_zone_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

In [ ]:
neighborhoods_venues_sorted.to_csv('neighborhoods_venues_sorted.csv', index=False)

Sixth step, format pickup time and normalize the data.

In [ ]:
pickup_hour = trip_data.tpep_pickup_datetime.apply(lambda x: time.strptime(x,"%Y-%m-%d %H:%M:%S").tm_hour)
pickup_hour.shape

In [ ]:
pickup_weekday = trip_data.tpep_pickup_datetime.apply(lambda x: time.strptime(x,"%Y-%m-%d %H:%M:%S").tm_wday)
pickup_weekday.shape

In [ ]:
trip_data.insert(loc=2,column='lpep_pickup_hour',value=pickup_hour)
trip_data.insert(loc=2,column='lpep_pickup_wday',value=pickup_weekday)
trip_data.head()

In [ ]:
trip_pickup_group = trip_data.groupby(['PULocationID','lpep_pickup_hour']).count()
trip_pickup_group.head()

In [ ]:
trip_pickup = pd.DataFrame(data={'0':[],'1':[],'2':[],'3':[],'4':[],'5':[],'6':[],'7':[],'8':[],'9':[],'10':[],'11':[],'12':[],'13':[],'14':[],'15':[],'16':[],'17':[],'18':[],'19':[],'20':[],'21':[],'22':[],'23':[]})

In [ ]:
for i in range(1,266):
    t = trip_pickup_group.query('PULocationID == '+str(i))
    s = 0
    for j in range(0,23):
        tt = t.query('lpep_pickup_hour == '+str(j))
        if tt.empty:
            ttt = 0
        else:
            ttt = tt.VendorID.values[0]
        s = s + ttt
    ss = []
    for j in range(0,24):
        tt = t.query('lpep_pickup_hour == '+str(j))
        if tt.empty or s == 0:
            ss.append(0.0)
        else:
            ss.append(tt.VendorID.values[0]*1.0/s)
    gtp = pd.DataFrame(data={'0':ss[0],'1':ss[1],'2':ss[2],'3':ss[3],'4':ss[4],'5':ss[5],'6':ss[6],'7':ss[7],'8':ss[8],'9':ss[9]
                             ,'10':ss[10],'11':ss[11],'12':ss[12],'13':ss[13],'14':ss[14],'15':ss[15],'16':ss[16],'17':ss[17]
                             ,'18':ss[18],'19':ss[19],'20':ss[20],'21':ss[21],'22':ss[22],'23':ss[23]}, index=[i])
    trip_pickup = trip_pickup.append(gtp,ignore_index=True)

trip_pickup.to_csv('trip_pickup.csv', index=False)
trip_pickup.head()